<a href="https://colab.research.google.com/github/pavlicL/NMDU/blob/main/NMDU_projekt_Lorena_Pavlic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Task
Razviti model za klasifikaciju jela s fotografija koristeći konvolucijske neuronske mreže (CNN) na Food-101 datasetu ("https://www.vision.ee.ethz.ch/datasets_extra/food-101/"). Implementirati i trenirati vlastiti CNN model te koristiti i unaprijed istrenirani ResNet50 model kroz transferno učenje. Evaluirati i usporediti performanse oba pristupa.

## Preuzimanje i priprema podataka

### Subtask:
Preuzmite Food-101 dataset i pripremite ga za korištenje u modelu (npr. podjela na setove za treniranje, validaciju i testiranje, promjena veličine slika, normalizacija).


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

# Download the dataset
dataset_name = "food101"
(train_data, test_data), ds_info = tfds.load(
    dataset_name,
    split=['train', 'validation'], # Use 'validation' for testing as per tfds documentation
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# The data is already split into train and test (validation in this case) and is in a format ready for use with TensorFlow.
# No need for manual extraction and organization.

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess_image(image, label):
    """Resizes and normalizes images."""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    return image, label

# Apply preprocessing to the datasets
train_data = train_data.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset preprocessing and batching complete.")

Dataset preprocessing and batching complete.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Define the number of classes (Food-101 has 101 classes)
num_classes = 101

# Initialize the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Optional dropout layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        13,029 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,181,989 (42.66 MB)

 Trainable params: 11,181,989 (42.66 MB)

 Non-trainable params: 0 (0.00 B)

## Treniranje vlastitog cnn modela

### Subtask:
Trenirajte vlastiti CNN model na pripremljenom datasetu.


In [ ]:
# 1) Kompajliraj model sa sparse loss-om
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 2) Treniraš točno onako kako si prije radio
epochs = 2
history = model.fit(
    train_data,       # tf.data.Dataset koji daje (x, y_int)
    epochs=epochs,
    validation_data=test_data
)


Epoch 1/2
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 198s 79ms/step - accuracy: 0.0145 - loss: 4.6056 - val_accuracy: 0.0553 - val_loss: 4.2818
Epoch 2/2
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 229s 93ms/step - accuracy: 0.0498 - loss: 4.3050 - val_accuracy: 0.0930 - val_loss: 4.0407


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import tensorflow_datasets as tfds

# ─── Parametri ────────────────────────────────────────────────────────────────
IMG_SIZE    = 224        # dimenzija slike
BATCH_SIZE  = 32         # batch size
NUM_CLASSES = 101        # broj klasa u Food-101
AUTOTUNE    = tf.data.AUTOTUNE
DATASET_NAME = 'food101'

# ─── 1) Učitaj dataset iz TFDS ────────────────────────────────────────────────
(train_data, test_data), ds_info = tfds.load(
    DATASET_NAME,
    split=['train', 'validation'],  # 'validation' je test skup prema TFDS
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Provjeri oblik sirovog zapisa (jedan primjer)
for img, lbl in train_data.take(1):
    print("Raw sample shapes:", img.shape, lbl.shape)
# očekivano: (224,224,3)  ()

# ─── 2) Preprocessing + jedno batchanje ──────────────────────────────────────
def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, depth=NUM_CLASSES)
    return image, label

def make_pipeline(ds, shuffle=False):
    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(BATCH_SIZE)    # ← batch točno jednom
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = make_pipeline(train_data, shuffle=True)
test_ds  = make_pipeline(test_data, shuffle=False)

# Provjeri oblik batchanog dataseta
print("Batched element_spec:", train_ds.element_spec)
for x, y in train_ds.take(1):
    print("Batched sample shapes:", x.shape, y.shape)
# očekivano: (BATCH_SIZE,224,224,3)  (BATCH_SIZE,NUM_CLASSES)

# ─── 3) Definicija i kompajliranje modela ───────────────────────────────────
model = models.Sequential([
    layers.Input((IMG_SIZE, IMG_SIZE, 3)),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(128,3, activation='relu', padding='same'),
    layers.MaxPool2D((2,2)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

# ─── 4) Treniranje ───────────────────────────────────────────────────────────
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=1,      # smanjeno za brzo testiranje
    verbose=2
)


Raw sample shapes: (512, 512, 3) ()
Batched element_spec: (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 101), dtype=tf.float32, name=None))
Batched sample shapes: (32, 224, 224, 3) (32, 101)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    25,690,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        25,957 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,809,573 (98.46 MB)

 Trainable params: 25,809,573 (98.46 MB)

 Non-trainable params: 0 (0.00 B)

2368/2368 - 240s - 101ms/step - accuracy: 0.0089 - loss: 4.6178 - val_accuracy: 0.0099 - val_loss: 4.6152


In [ ]:
import tensorflow as tf

def preprocess_image_with_one_hot(image, label):
    """Resizes and normalizes images and one-hot encodes labels."""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    label = tf.one_hot(label, depth=101) # One-hot encode the label
    return image, label

# Re-load the dataset to ensure correct initial state before mapping and batching
# Assuming the original dataset loading provided `ds_train` and `ds_test`
# If not, you might need to re-execute the initial loading step
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Apply preprocessing with one-hot encoding and then batch and prefetch
train_data = ds_train.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = ds_test.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.")

# Train the model again with the corrected data
epochs = 1 # Specify the number of epochs
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 220s 93ms/step - accuracy: 0.0107 - loss: 4.6156 - val_accuracy: 0.0099 - val_loss: 4.6152


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds

# Clear the Keras session to reset the state
tf.keras.backend.clear_session()

# Define the number of classes (Food-101 has 101 classes)
num_classes = 101

# Redefine the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Optional dropout layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

print("Keras session cleared, model redefined and recompiled.")

# Define the preprocessing function with one-hot encoding
def preprocess_image_with_one_hot(image, label):
    """Resizes and normalizes images and one-hot encodes labels."""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    label = tf.one_hot(label, depth=num_classes) # One-hot encode the label
    return image, label

# Re-load the dataset to ensure correct initial state before mapping and batching
(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

# Apply preprocessing with one-hot encoding and then batch and prefetch
train_data = ds_train.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = ds_test.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.")

# Train the model again with the corrected data
epochs = 1 # Specify the number of epochs
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Keras session cleared, model redefined and recompiled.
Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 232s 95ms/step - accuracy: 0.0094 - loss: 4.6327 - val_accuracy: 0.0099 - val_loss: 4.6152


## Evaluacija vlastitog CNN modela

### Subtask:
Evaluirajte performanse vlastitog modela koristeći testni set.

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

790/790 ━━━━━━━━━━━━━━━━━━━━ 43s 54ms/step - accuracy: 0.0108 - loss: 4.6150
Test Loss: 4.6152
Test Accuracy: 0.0099


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Define the number of classes (Food-101 has 101 classes)
num_classes = 101


# Initialize the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Optional dropout layer
    Dense(num_classes, activation='softmax')
])

(ds_train, ds_test), ds_info = tfds.load(
    'food101',
    split=['train', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

def preprocess(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0

    return image, label

train_data = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


test_data = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Compile the model with sparse loss
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # Using sparse loss for integer labels
    metrics=['accuracy']
)

model.summary()

epochs = 1
history = model.fit(
    train_data,
    epochs=epochs,
    validation_data=test_data
)

print("Model defined, compiled, and training started.")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_72 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_72 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_73 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_73 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_74 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_74 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_24 (Flatten)            │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 101)            │        13,029 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,181,989 (42.66 MB)

 Trainable params: 11,181,989 (42.66 MB)

 Non-trainable params: 0 (0.00 B)

2368/2368 ━━━━━━━━━━━━━━━━━━━━ 200s 82ms/step - accuracy: 0.0135 - loss: 4.6118 - val_accuracy: 0.0483 - val_loss: 4.3812
Model defined, compiled, and training started.


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

# Download the dataset
dataset_name = "food101"
(train_data_raw, test_data_raw), ds_info = tfds.load(
    dataset_name,
    split=['train', 'validation'], # Use 'validation' for testing as per tfds documentation
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# The data is already split into train and test (validation in this case) and is in a format ready for use with TensorFlow.
# No need for manual extraction and organization.

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
num_classes = ds_info.features['label'].num_classes # Get the number of classes from the dataset info

def preprocess_image(image, label):
    """Resizes and normalizes images."""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    return image, label

# Apply preprocessing to the datasets
train_data = train_data_raw.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data_raw.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset preprocessing and batching complete.")

## Implementacija vlastitog cnn modela

### Subtask:
Definirajte arhitekturu vlastite konvolucijske neuronske mreže.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Initialize the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Optional dropout layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), # Use SparseCategoricalCrossentropy for integer labels
              metrics=['accuracy'])

model.summary()

## Treniranje vlastitog cnn modela

### Subtask:
Trenirajte vlastiti CNN model na pripremljenom datasetu.

In [ ]:
# Train the model
epochs = 10
history = model.fit(
    train_data,       # tf.data.Dataset koji daje (x, y_int)
    epochs=epochs,
    validation_data=test_data
)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

# Download the dataset
dataset_name = "food101"
(train_data_raw, test_data_raw), ds_info = tfds.load(
    dataset_name,
    split=['train', 'validation'], # Use 'validation' for testing as per tfds documentation
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# The data is already split into train and test (validation in this case) and is in a format ready for use with TensorFlow.
# No need for manual extraction and organization.

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
num_classes = ds_info.features['label'].num_classes # Get the number of classes from the dataset info

def preprocess_image_with_one_hot(image, label):
    """Resizes and normalizes images and one-hot encodes labels."""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    label = tf.one_hot(label, depth=num_classes) # One-hot encode the label
    return image, label

# Apply preprocessing with one-hot encoding and then batch and prefetch
train_data = train_data_raw.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data_raw.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.")

## Implementacija vlastitog cnn modela

### Subtask:
Definirajte arhitekturu vlastite konvolucijske neuronske mreže.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Clear the Keras session to reset the state
tf.keras.backend.clear_session()

# Initialize the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Optional dropout layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(), # Use CategoricalCrossentropy for one-hot encoded labels
              metrics=['accuracy'])

model.summary()

## Treniranje vlastitog cnn modela

### Subtask:
Trenirajte vlastiti CNN model na pripremljenom datasetu.

In [ ]:
# Train the model
epochs = 10 # Specify the number of epochs
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

## Evaluacija vlastitog CNN modela

### Subtask:
Evaluirajte performanse vlastitog modela koristeći testni set.

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

790/790 ━━━━━━━━━━━━━━━━━━━━ 43s 54ms/step - accuracy: 0.0922 - loss: 4.0308
Test Loss: 4.0407
Test Accuracy: 0.0930


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os

# Download the dataset
dataset_name = "food101"
(train_data_raw, test_data_raw), ds_info = tfds.load(
    dataset_name,
    split=['train', 'validation'], # Use 'validation' for testing as per tfds documentation
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# The data is already split into train and test (validation in this case) and is in a format ready for use with TensorFlow.
# No need for manual extraction and organization.

**Reasoning**:
Implement image transformations, including resizing and normalization, and prepare data loaders for efficient data loading.

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
num_classes = ds_info.features['label'].num_classes # Get the number of classes from the dataset info

def preprocess_image_with_one_hot(image, label):
    """Resizes and normalizes images and one-hot encodes labels."""
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.cast(image, tf.float32) / 255.0 # Normalize to [0, 1]
    label = tf.one_hot(label, depth=num_classes) # One-hot encode the label
    return image, label

# Apply preprocessing with one-hot encoding and then batch and prefetch
train_data = train_data_raw.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data_raw.map(preprocess_image_with_one_hot).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.")

Dataset reloaded, preprocessed with one-hot encoding, batched, and prefetched.


## Implementacija vlastitog cnn modela

### Subtask:
Definirajte arhitekturu vlastite konvolucijske neuronske mreže.

**Reasoning**:
Import necessary Keras layers and models, initialize a sequential model, add convolutional and pooling layers, a flatten layer, dense layers, and compile the model with an optimizer, loss function, and metrics.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

# Clear the Keras session to reset the state
tf.keras.backend.clear_session()

# Initialize the Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Optional dropout layer
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(), # Use CategoricalCrossentropy for one-hot encoded labels
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        13,029 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,181,989 (42.66 MB)

 Trainable params: 11,181,989 (42.66 MB)

 Non-trainable params: 0 (0.00 B)

## Treniranje vlastitog cnn modela

### Subtask:
Trenirajte vlastiti CNN model na pripremljenom datasetu.

**Reasoning**:
Train the compiled custom CNN model using the `fit` method with the prepared datasets and a specified number of epochs.

In [ ]:
# Train the model
epochs = 10 # Specify the number of epochs
history = model.fit(train_data,
                    epochs=epochs,
                    validation_data=test_data)

Epoch 1/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 224s 93ms/step - accuracy: 0.0195 - loss: 4.5584 - val_accuracy: 0.0396 - val_loss: 4.3565
Epoch 2/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 258s 92ms/step - accuracy: 0.0443 - loss: 4.3249 - val_accuracy: 0.0692 - val_loss: 4.1300
Epoch 3/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 178s 75ms/step - accuracy: 0.0657 - loss: 4.1478 - val_accuracy: 0.1030 - val_loss: 3.9343
Epoch 4/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 204s 76ms/step - accuracy: 0.1011 - loss: 3.9107 - val_accuracy: 0.1335 - val_loss: 3.7516
Epoch 5/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 220s 93ms/step - accuracy: 0.1454 - loss: 3.6307 - val_accuracy: 0.1437 - val_loss: 3.6683
Epoch 6/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 179s 76ms/step - accuracy: 0.1972 - loss: 3.3123 - val_accuracy: 0.1467 - val_loss: 3.6861
Epoch 7/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 201s 75ms/step - accuracy: 0.2528 - loss: 2.9979 - val_accuracy: 0.1413 - val_loss: 3.7991
Epoch 8/10
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 201s 75ms/step - accuracy: 